<a href="https://colab.research.google.com/github/theDikshaSaxena/Recontextualization-Detection/blob/main/IntentDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
%env CUDA_LAUNCH_BLOCKING=1
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
! pip install torchtext==0.10.1

import torch
device = torch.device("cuda")
torch.cuda.init()
import numpy as np
import time
import math

env: CUDA_LAUNCH_BLOCKING=1
ERROR: Could not find a version that satisfies the requirement torchtext==0.10.1 (from versions: 0.1.1, 0.2.0, 0.2.1, 0.2.3, 0.3.1, 0.4.0, 0.5.0, 0.6.0, 0.12.0, 0.13.0, 0.13.1, 0.14.0, 0.14.1, 0.15.1, 0.15.2)
ERROR: No matching distribution found for torchtext==0.10.1


RuntimeError: ignored

In [6]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.1 MB/s eta 0:00:00


In [8]:
from google.colab import drive
drive.mount('/content/gdrive')

import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from torch.utils.data import DataLoader, TensorDataset

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [9]:
# Load Pre-trained BERT Model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3) # 3 labels: "demoralizing", "calling to action", "none"

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
# Read the labeled dataset from CSV file
dataset_path = '/content/gdrive/My Drive/RD/IntentDetection.csv'
df = pd.read_csv(dataset_path)

In [15]:
# Data Preparation
sentences = df['sentence'].tolist()
labels = df['label'].tolist()

inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
labels = torch.tensor(labels)

In [16]:
# Fine-tuning the Model
dataset = TensorDataset(inputs.input_ids, inputs.attention_mask, labels)
train_dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [33]:
from tqdm.notebook import tqdm

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

for epoch in range(30):
    model.train()
    progress_bar = tqdm(train_dataloader, desc=f'Epoch {epoch + 1}/{30}')
    for batch in progress_bar:
        batch = tuple(t.to(device) for t in batch)
        inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        progress_bar.set_postfix({'Loss': loss.item()})


Epoch 1/3:   0%|          | 0/10 [00:00<?, ?it/s]

Epoch 2/3:   0%|          | 0/10 [00:00<?, ?it/s]

Epoch 3/3:   0%|          | 0/10 [00:00<?, ?it/s]

Epoch 4/3:   0%|          | 0/10 [00:00<?, ?it/s]

Epoch 5/3:   0%|          | 0/10 [00:00<?, ?it/s]

...

Epoch 26/3:   0%|          | 0/10 [00:00<?, ?it/s]

Epoch 27/3:   0%|          | 0/10 [00:00<?, ?it/s]

Epoch 28/3:   0%|          | 0/10 [00:00<?, ?it/s]

Epoch 29/3:   0%|          | 0/10 [00:00<?, ?it/s]

Epoch 30/3:   0%|          | 0/10 [00:00<?, ?it/s]

In [34]:
# Save the fine-tuned model if needed
model.save_pretrained("/content/gdrive/My Drive/RD/fine_tuned_model_intent")

# Load the fine-tuned model
# model = BertForSequenceClassification.from_pretrained("fine_tuned_model")

In [39]:
model.eval()

test_sentence = "Its time for the world to stop these attacks now before its to late."
inputs = tokenizer(test_sentence, return_tensors="pt")
inputs.to(device)
with torch.no_grad():
    inputs.to(device)
    outputs = model(**inputs)

intent_id = torch.argmax(outputs.logits, dim=1)
predicted_intent = intent_id.item()
# print(predicted_intent)
label_map = {
    0: "Demoralizing",
    1: "Call to Action",
    2: "No Intent"
}
predicted_intent_label = label_map[predicted_intent]
print(predicted_intent_label)

Demoralizing
